In [1]:
!pip install langgraph langchain langchain-community langchain-openai openai chromadb tiktoken unstructured serpapi
!pip install chromadb llama-index llama-index-vector-stores-chroma fastembed transformers huggingface_hub torch gradio
!pip install llama-index-embeddings-fastembed
!pip install smolagents
!pip install pymongo dnspython google-search-results
!pip install chromadb \
    llama-index \
    llama-index-vector-stores-chroma \
    llama-index-embeddings-fastembed \
    llama-index-llms-huggingface \
    fastembed \
    transformers \
    huggingface_hub \
    torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Authenticate Hugging Face (if needed)
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool, CodeAgent
from typing import Dict, TypedDict, List
from langgraph.graph import END, StateGraph
from llama_index.core import QueryBundle, VectorStoreIndex, StorageContext, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.workflow import AgentWorkflow, ReActAgent
import chromadb
import torch
from transformers import AutoTokenizer
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from smolagents import GoogleSearchTool
import os
from pymongo import MongoClient
from datetime import datetime
import uuid
from urllib.parse import quote_plus
import json  # Added missing import

In [4]:
# Set up Serper API key
os.environ["SERPER_API_KEY"] = "2b31aea7a2305eaf1d86843679d302b736f3556d"
# 1. Load documents
documents = SimpleDirectoryReader("/content/Data").load_data()

# 2. Initialize embedding model
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

# 3. Initialize LLM (Mistral-7B-Instruct)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("</s>"),
]

llm = HuggingFaceLLM(
    context_window=8192,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt="You are a Q&A assistant. Your goal is to answer questions accurately based on the context provided.",
    query_wrapper_prompt="[INST] {query_str} [/INST]",
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    stopping_ids=stopping_ids,
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float16}
)

# 4. Initialize ChromaDB and create index
chroma_client = chromadb.PersistentClient(path="./chroma_db")
vector_store = ChromaVectorStore(chroma_collection=chroma_client.get_or_create_collection("my_collection"))
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=embed_model
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
import json  # Added missing import
# Define State
class AgentState(TypedDict):
    question: str
    generation: str
    source: str  # "documents" or "web"
    decision_reason: str

has_documents = False  # Set to True if you have documents
# Improved Web Search Tool
def search_web(query: str) -> str:
    """Search the web for current information"""
    try:
        search_tool = GoogleSearchTool(provider="serper")
        results = search_tool(query)

        # Handle case where results is a string (error message)
        if isinstance(results, str):
            return results

        # Process results safely
        formatted = []
        for res in results[:3]:  # Get top 3 results
            title = res.get('title', 'No title')
            link = res.get('link', 'No link')
            snippet = res.get('snippet', 'No snippet')
            formatted.append(f"Title: {title}\nLink: {link}\nSnippet: {snippet}")

        return "\n\n".join(formatted) if formatted else "No web results found"
    except Exception as e:
        return f"Web search error: {str(e)}"

# Improved Document Search Tool
def search_documents(query: str) -> str:
    """Search through available documents"""
    # if not has_documents:
    #     return "No documents available"
    try:
        query_engine = index.as_query_engine(llm=llm)
        response = query_engine.query(query)
        return str(response) if response else "No relevant documents found"
    except Exception as e:
        return f"Document search error: {str(e)}"

# Enhanced Decision Agent
class ToolDecisionAgent:
    def __init__(self, llm):
        self.llm = llm
        self.rules = """Decision Rules:
        1. Use documents for:
           - Relation marketing
        2. Use web for:
           - Current events/trends"""

    def decide_tool(self, question: str) -> Dict:
        """Make intelligent tool selection"""
        prompt = f"""Analyze this question and decide whether to search documents or web:

        Question: {question}

        {self.rules}

        Return JSON format: {{"tool": "documents"|"web", "reason": "explanation"}}"""

        try:
            response = self.llm.complete(prompt)
            # Safely parse JSON response
            if response.text.strip().startswith("{"):
                return json.loads(response.text)
            return {"tool": "web", "reason": "default fallback"}
        except Exception as e:
            print(f"Decision error: {str(e)}")
            return {"tool": "web", "reason": "error fallback"}

# Create Nodes
def decide_tool(state: AgentState) -> Dict:
    """Node: Decide which tool to use"""
    decision_agent = ToolDecisionAgent(llm)
    decision = decision_agent.decide_tool(state["question"])
    print(f"Decision: {decision['tool']} ({decision['reason']})")
    return {
        "source": decision["tool"],
        "decision_reason": decision["reason"]
    }

def execute_search(state: AgentState) -> Dict:
    """Node: Execute the chosen search"""
    question = state["question"]
    source = state["source"]

    print(f"---SEARCHING {source.upper()}---")
    result = search_documents(question) if source == "documents" else search_web(question)

    # Handle empty results
    if "no " in result.lower() or "error" in result.lower():
        print(f"Warning: {result}")
        if source == "documents":
            print("Falling back to web search")
            result = search_web(question)
            source = "web"

    return {
        "generation": result,
        "source": source
    }

# Create Workflow
workflow = StateGraph(AgentState)

# Add Nodes
workflow.add_node("decide_tool", decide_tool)
workflow.add_node("execute_search", execute_search)

# Build Graph
workflow.set_entry_point("decide_tool")
workflow.add_edge("decide_tool", "execute_search")
workflow.add_edge("execute_search", END)

# Compile
app = workflow.compile()

# Main Loop
def run_chat():
    print("\nIntelligent Research Assistant Initialized")
    print("Type your question or 'exit' to quit\n")

    while True:
        try:
            user_input = input("Question: ").strip()

            if user_input.lower() in ['exit', 'quit']:
                print("\nGoodbye!")
                break

            if not user_input:
                continue

            # Initialize state
            init_state = {
                "question": user_input,
                "generation": "",
                "source": "",
                "decision_reason": ""
            }

            # Execute workflow
            result = app.invoke(init_state)

            # Format output
            print(f"\nAnswer (from {result['source']}):")
            print(result["generation"])
            print(f"\nReason: {result['decision_reason']}")
            print("---\n")

        except KeyboardInterrupt:
            print("\nGoodbye!")
            break
        except Exception as e:
            print(f"\nError: {str(e)}")
            print("Please try another question.\n")

if __name__ == "__main__":
    run_chat()


Intelligent Research Assistant Initialized
Type your question or 'exit' to quit

Question: What is relationship marketing? 


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Decision: documents (explanation)
---SEARCHING DOCUMENTS---

Answer (from documents):
Relationship marketing is a marketing strategy that focuses on building long-term engagement and loyalty with customers, rather than just acquiring new ones. It emphasizes the importance of building trust and personalized experiences with customers to foster long-term relationships and increase customer retention, satisfaction, and lifetime value.

Reason: explanation
---

Question: How can I RETAIN MORE CUSTOMERS


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Decision: documents (Relation marketing)
---SEARCHING DOCUMENTS---

Answer (from documents):
To retain more customers, you can focus on relationship marketing strategies that emphasize long-term customer engagement and loyalty. Some effective strategies include personalized experiences, loyalty programs, excellent customer service, and consistent engagement. Building trust with customers through transparency, consistent service, and personalized interactions can also help retain them. Additionally, offering surprise and delight strategies, such as unexpected rewards, birthday gifts, or exclusive offers, can increase emotional connection and loyalty.

Reason: Relation marketing
---

Question: suggest me some infuencers to help me get better custormer retention


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Decision: web (default fallback)
---SEARCHING WEB---

Answer (from web):
## Search Results
0. [The top customer experience influencers to follow in 2023](https://www.cxnetwork.com/cx-experience/articles/cx-networks-top-influencers-to-follow-in-2023)
Date published: Mar 7, 2023

Our list includes CEOs, media personalities, CXOs and thought leaders, all of whom are making a significant impact in the world of customer experience.

1. [6 Customer Service Influencers Give Their Best Advice - Formilla Blog](https://blog.formilla.com/customer-service-influencers/)
Date published: Feb 26, 2017

Bill Quiseng is a B2C customer service influencer and frequently tweets about customer experience. Bill, what is the hardest problem in a ...

2. [Our Top Customer Success Influencers of 2024 - Custify Blog](https://www.custify.com/blog/top-customer-success-influencers-of-2024/)
Date published: Dec 20, 2024

Discover our top 75 Customer Success Influencers of 2024, sharing their best strategies on onboa

**Memory integration attempt**

In [36]:
from pymongo import MongoClient
from datetime import datetime
import uuid
from urllib.parse import quote_plus

# MongoDB Setup - Replace with your actual credentials
username = "mouhebdemacia1"
password = "nobodycan@2H"  # Special characters will be encoded
cluster = "cluster0.a2p7opk.mongodb.net"

# URL encode the password
encoded_password = quote_plus(password)
MONGODB_URI = f"mongodb+srv://{username}:{encoded_password}@{cluster}/conversation_db"
# Initialize MongoDB client
try:
    client = MongoClient(MONGODB_URI)
    db = client["conversation_db"]
    conversations = db["conversation_history"]
    print("Successfully connected to MongoDB!")
except Exception as e:
    print(f"Failed to connect to MongoDB: {str(e)}")
    # Fallback to in-memory storage if MongoDB fails
    conversations = None
    conversation_memory = []

# Enhanced Web Search Tool
def search_web(query: str) -> str:
    """Search the web for current information"""
    try:
        search_tool = GoogleSearchTool(provider="serper")
        results = search_tool(query)

        # Handle case where results is a string (error message)
        if isinstance(results, str):
            return results

        # Process results safely
        formatted = []
        for res in results[:3]:  # Get top 3 results
            title = res.get('title', 'No title')
            link = res.get('link', 'No link')
            snippet = res.get('snippet', 'No snippet')
            formatted.append(f"Title: {title}\nLink: {link}\nSnippet: {snippet}")

        return "\n\n".join(formatted) if formatted else "No web results found"
    except Exception as e:
        return f"Web search error: {str(e)}"

# Enhanced Document Search Tool
def search_documents(query: str) -> str:
    """Search through available documents"""
    try:
        query_engine = index.as_query_engine(llm=llm)
        response = query_engine.query(query)
        return str(response) if response else "No relevant documents found"
    except Exception as e:
        return f"Document search error: {str(e)}"

# Robust Decision Agent with Memory Awareness
class ToolDecisionAgent:
    def __init__(self, llm):
        self.llm = llm
        self.rules = """Decision Rules:
        1. Use documents for:
           - Relation marketing
        2. Use web for:
           - Current events/trends
           """

    def decide_tool(self, question: str, conversation_id: str) -> Dict:
        """Make intelligent tool selection with conversation context"""
        # Check conversation history
        history = []
        try:
            if conversations is not None:  # Fixed collection check
                history = list(conversations.find(
                    {"conversation_id": conversation_id},
                    {"question": 1, "source": 1, "_id": 0}
                ).sort("timestamp", -1).limit(3))
            elif hasattr(conversation_memory, '__iter__'):
                history = [m for m in conversation_memory if m["conversation_id"] == conversation_id][-3:]
        except Exception as e:
            print(f"History retrieval error: {str(e)}")

        prompt = f"""Analyze this question and decide whether to search documents or web:

        Current Question: {question}

        Recent Conversation History:
        {json.dumps(history, indent=2) if history else 'No recent history'}

        {self.rules}

        Return JSON format: {{"tool": "documents"|"web", "reason": "explanation"}}"""

        try:
            response = self.llm.complete(prompt)
            if response.text.strip().startswith("{"):
                return json.loads(response.text)
            return {"tool": "web", "reason": "default fallback"}
        except Exception as e:
            print(f"Decision error: {str(e)}")
            return {"tool": "web", "reason": "error fallback"}

# Define State with Conversation ID
class AgentState(TypedDict):
    question: str
    generation: str
    source: str
    decision_reason: str
    conversation_id: str

# Create Nodes with MongoDB Integration
def start_conversation(state: AgentState) -> Dict:
    """Initialize or continue conversation"""
    if not state.get("conversation_id"):
        state["conversation_id"] = str(uuid.uuid4())
    return state

def decide_tool(state: AgentState) -> Dict:
    """Node: Decide which tool to use with conversation context"""
    decision_agent = ToolDecisionAgent(llm)
    decision = decision_agent.decide_tool(state["question"], state["conversation_id"])
    print(f"Decision: {decision['tool']} ({decision['reason']})")
    return {
        "source": decision["tool"],
        "decision_reason": decision["reason"]
    }

def execute_search(state: AgentState) -> Dict:
    """Node: Execute the chosen search"""
    question = state["question"]
    source = state["source"]

    print(f"---SEARCHING {source.upper()}---")
    result = search_documents(question) if source == "documents" else search_web(question)

    # Handle empty results
    if "no " in result.lower() or "error" in result.lower():
        print(f"Warning: {result}")
        if source == "documents":
            print("Falling back to web search")
            result = search_web(question)
            source = "web"

    return {
        "generation": result,
        "source": source
    }

def save_to_memory(state: AgentState) -> Dict:
    """Node: Save conversation to memory"""
    record = {
        "conversation_id": state["conversation_id"],
        "question": state["question"],
        "answer": state["generation"],
        "source": state["source"],
        "decision_reason": state["decision_reason"],
        "timestamp": datetime.now()
    }

    try:
        if conversations is not None:  # Fixed collection check
            conversations.insert_one(record)
        elif hasattr(conversation_memory, 'append'):
            conversation_memory.append(record)
    except Exception as e:
        print(f"Failed to save to memory: {str(e)}")

    return state

# Create Workflow
workflow = StateGraph(AgentState)

# Add Nodes
workflow.add_node("start_conversation", start_conversation)
workflow.add_node("decide_tool", decide_tool)
workflow.add_node("execute_search", execute_search)
workflow.add_node("save_to_memory", save_to_memory)

# Build Graph
workflow.set_entry_point("start_conversation")
workflow.add_edge("start_conversation", "decide_tool")
workflow.add_edge("decide_tool", "execute_search")
workflow.add_edge("execute_search", "save_to_memory")
workflow.add_edge("save_to_memory", END)

# Compile
app = workflow.compile()

# Main Loop with Conversation Memory
def run_chat():
    print("\nIntelligent Research Assistant with Memory Initialized")
    print("Type your question or 'exit' to quit\n")

    conversation_id = None

    while True:
        try:
            user_input = input("Question: ").strip()

            if user_input.lower() in ['exit', 'quit']:
                print("\nGoodbye!")
                break

            if not user_input:
                continue

            # Initialize state
            result = app.invoke({
                "question": user_input,
                "generation": "",
                "source": "",
                "decision_reason": "",
                "conversation_id": conversation_id
            })

            # Store conversation ID for next turn
            conversation_id = result["conversation_id"]

            # Format output
            print(f"\nAnswer (from {result['source']}):")
            print(result["generation"])
            print(f"\nReason: {result['decision_reason']}")
            print("---\n")

        except KeyboardInterrupt:
            print("\nGoodbye!")
            break
        except Exception as e:
            print(f"\nError: {str(e)}")
            print("Please try another question.\n")

if __name__ == "__main__":
    run_chat()

NameError: name 'encodeURIComponent' is not defined

In [41]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from urllib.parse import quote_plus

# MongoDB Configuration
username = "mouhebdemacia1"
password = "nobodycan@2H"  # Contains special character @
cluster = "cluster0.a2p7opk.mongodb.net"
dbname = "conversation_db"

# Properly encode the password
encoded_password = quote_plus(password)

# Construct the connection URI
MONGODB_URI = f"mongodb+srv://{username}:{encoded_password}@{cluster}/{dbname}?retryWrites=true&w=majority&appName=Cluster0"

# Create a new client and connect to the server
client = MongoClient(MONGODB_URI, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")

    # Test database operations
    db = client[dbname]
    test_collection = db["test_connection"]
    test_collection.insert_one({"test": "connection", "timestamp": datetime.now()})
    print("Test document inserted successfully!")

except Exception as e:
    print(f"Connection failed: {e}")

Connection failed: SSL handshake failed: ac-mrzqo3k-shard-00-00.a2p7opk.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1016) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: ac-mrzqo3k-shard-00-02.a2p7opk.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1016) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: ac-mrzqo3k-shard-00-01.a2p7opk.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1016) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 67fdb846e4bc452463afc3c8, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-mrzqo3k-shard-00-00.a2p7opk.mongodb.net', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('SSL handshake failed: ac-mrzqo3k-shard-00-